In [2]:
import pickle

# Replace 'file.p' with the path to your .p file
file_path = 'files/project_spreads.p'

with open(file_path, 'rb') as file:
    data = pickle.load(file)

# Now, 'data' contains the deserialized object from the .p file
print(data)

{'IT': {'green':            XS2395580892 XS2455938212 XS2491189408 XS2026150313 XS2534976886  \
DATE                                                                          
2014-01-31          NaN          NaN          NaN          NaN          NaN   
2014-02-03          NaN          NaN          NaN          NaN          NaN   
2014-02-04          NaN          NaN          NaN          NaN          NaN   
2014-02-05          NaN          NaN          NaN          NaN          NaN   
2014-02-06          NaN          NaN          NaN          NaN          NaN   
...                 ...          ...          ...          ...          ...   
2023-02-21     1.192561     1.489636     0.453896     1.252126     1.212571   
2023-02-22     1.037088     1.417657     0.369602     1.153246     1.111639   
2023-02-23     1.072811     1.444528     0.376017     1.132925     1.085191   
2023-02-24     1.189179     1.560091      0.50654     1.249678     1.215543   
2023-02-27     1.096786     1.46082